<div  align="center">
    <img src="./files/Waterloo.gif" width="20%" height="20%"> 
</div>

<div align="center">
    <font size='6' color='blue'><b>Recurrent Neural Network</b></font><br/>
    <font size='3' color='blue'><b>(Code Example)</b></font><br/>
    <font size='3' color='blue'><b>Ali Ghodsi</b></font>
</div>


# IMDB Sentiment Classification Task

### Task Description:
<p>We want to perform sentiment analysis on movie reviews from the Large Movie Review Dataset, sometimes known as the IMDB dataset. In this task, given a movie review, the model attempts to predict whether it is positive or negative. This is a binary classification task<sup><a href="#R1">[1]</a></sup>.</p>

### IMBD dataset in Keras:
<p>Dataset of 25,000 movies reviews from IMDB, labeled by sentiment (positive/negative). Reviews have been preprocessed, and each review is encoded as a sequence of word indexes (integers). For convenience, words are indexed by overall frequency in the dataset, so that for instance the integer "3" encodes the 3rd most frequent word in the data. This allows for quick filtering operations such as: "only consider the top 10,000 most common words, but eliminate the top 20 most common words".<sup><a href="#R2">[2]</a></sup></p>
(For more information visit reference <a href="#R2" target="_blank">[2]</a>.)
<br /><br />
<font color="red"><b>Note:</b> Most parts of the fallowing code are captured from <a href="#R3">[3]</a</font>

# Step 1)  Import packages and set parameters
In the following code except LSTM layer, another new layer which is called embedding Layer is used. Later it will be explained.


In [1]:
import keras
import numpy as np
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb

Using TensorFlow backend.


### Setting paramiters

In [2]:
max_features = 20000
maxlen = 80  # cut texts after this number of words (among top max_features most common words)
batch_size = 32

# Step 2) Load and prepare data

In [3]:
print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

Loading data...
25000 train sequences
25000 test sequences


### Pad sequences
Pad the word sequences in each sample. 

In [4]:
print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

Pad sequences (samples x time)
x_train shape: (25000, 80)
x_test shape: (25000, 80)


# Step 3) Model Definition
<b>Embedding layer:</b> Turns positive integers (indexes) into dense vectors of fixed size. eg. [[4], [20]] -> [[0.25, 0.1], [0.6, -0.2]]<sup><a href="#R4" target="_blank">[4]</a></sup> This layer can only be used as the first layer in a model.

In [5]:
print('Build model...')
model = Sequential()

#######################
#  Embedding layer[4] #
#######################
# - the model will take as input an integer matrix of size (batch_size, input_length).
# - the largest integer (i.e. word index) in the input should be no larger than (max_features-1) 
#   (vocabulary size).
# - now model.output_shape == (None, input_length, 128), where None is the batch dimension.
model.add(Embedding(max_features, 128, input_length=maxlen))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))    

Build model...


### Summary of model

In [6]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 80, 128)           2560000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 2,691,713
Trainable params: 2,691,713
Non-trainable params: 0
_________________________________________________________________


# Step 4) Compiling model

In [6]:
# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
metrics=['accuracy'])

# Step 5) Learning model and fit it on training data

In [7]:
print('Train...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=15,
validation_data=(x_test, y_test))

Train...
Train on 25000 samples, validate on 25000 samples
Epoch 1/15
25000/25000 [==============================] - 220s - loss: 0.4589 - acc: 0.7848 - val_loss: 0.3808 - val_acc: 0.8359
Epoch 2/15
25000/25000 [==============================] - 213s - loss: 0.2973 - acc: 0.8795 - val_loss: 0.4464 - val_acc: 0.8312
Epoch 3/15
25000/25000 [==============================] - 217s - loss: 0.2146 - acc: 0.9180 - val_loss: 0.4146 - val_acc: 0.8340
Epoch 4/15
25000/25000 [==============================] - 218s - loss: 0.1538 - acc: 0.9434 - val_loss: 0.4657 - val_acc: 0.8296
Epoch 5/15
25000/25000 [==============================] - 219s - loss: 0.1094 - acc: 0.9613 - val_loss: 0.5915 - val_acc: 0.8292
Epoch 6/15
25000/25000 [==============================] - 217s - loss: 0.0781 - acc: 0.9742 - val_loss: 0.6407 - val_acc: 0.8118
Epoch 7/15
25000/25000 [==============================] - 216s - loss: 0.0594 - acc: 0.9792 - val_loss: 0.7037 - val_acc: 0.8212
Epoch 8/15
25000/25000 [==============

## Save and load model

#### save model

In [ ]:
# This function saves the model in 'rnn.h5' file
#Note: Uncomment it in the case of saving the trained model.

# model.save('./rnn.h5')

#### load model

In [10]:
# You can load a pretrained model by using this function.
model = keras.models.load_model('./rnn.h5')

## Evaluate and predict labels of test data

#### Evaluate model

In [17]:
print('Evaluating model...')
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('\n\nTest score:', score)
print('Test accuracy:', acc)

Evaluating model...
24992/25000 [============================>.] - ETA: 0s

Test score: 1.14194533084
Test accuracy: 0.80828


#### Predict labels

In [19]:
y_prd = model.predict(x_test)

y_prd = [1 if v > 0.5 else 0 for v in y_prd]
print('First ten predicted label and true label of test data')
print(np.array(y_prd[0:10]))
print(y_test[0:10])

# Alternative way for evaluating model
print('\nEvaluating model (alternative way)...')
nb_correct_labels = np.sum(y_prd == y_test)
print('Test accuracy: ', nb_correct_labels/len(y_test))


First ten predicted label and true label of test data
[1 0 1 1 0 1 0 0 1 1]
[1 1 1 1 1 0 0 0 1 1]

Evaluating model (alternative way)...
Test accuracy:  0.80828


### References
<b id="R1">[1]</b> http://deeplearning.net/tutorial/lstm.html <br />
<b id="R2">[2]</b> https://keras.io/datasets/<br />
<b id="R3">[3]</b> https://github.com/keras-team/keras/blob/master/examples/imdb_lstm.py<br />
<b id="R4">[4]</b> https://keras.io/layers/embeddings/